In [122]:
import numpy as np
import pandas as pd
from math import log
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.linear_model import Ridge

import sys
sys.path.append('../utils')
from multiple_regression import *
from round_result import round_result


In [123]:
wine = pd.read_csv('../datasets/winequality-white.csv', sep=';')
wine['const'] = 1
wine['alcohol_log'] = wine['alcohol'].apply(lambda x: log(x))

X_train, X_test, y_train, y_test = train_test_split(wine.drop('quality', axis=1), wine['quality'], test_size=0.2, random_state=0)
wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,const,alcohol_log
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,1,2.174752
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,1,2.251292
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,1,2.312535
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,1,2.292535
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,1,2.292535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,1,2.415914
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,1,2.261763
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,1,2.240710
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,1,2.549445


In [124]:
y_test.describe()

count    980.000000
mean       5.821429
std        0.939296
min        3.000000
25%        5.000000
50%        6.000000
75%        6.000000
max        8.000000
Name: quality, dtype: float64

Zanim przejdziemy do pracy nad modelem, obserwujemy zakresy uzyskane przez podział danych.

In [125]:
hypothesis_features = ['const', 'alcohol', 'volatile acidity', 'density', 'alcohol_log']
considered_features = []

test_results = []

Będziemy iterować po kolejnych cechach z hipotezy, dodając je kolejno do modelu i obserwując zachodzące w nim zmiany.

In [126]:
# const
i = 0
considered_features.append(hypothesis_features[i])
test_results.append(describe_multiple_regression_model(X_train, X_test, y_train, y_test, considered_features, hypothesis_features[i]))

y_pred = pd.DataFrame(test_results[i][1], index=y_test.index, columns=["y_pred"])
print(y_pred)
y_train.describe()

Added: const
	Avg train error =	0.7582104012683419
	Avg validate error =	0.7616538590496968
	Test error =		0.886363066388164

        y_pred
2762  5.892037
42    5.892037
1419  5.892037
3664  5.892037
2125  5.892037
...        ...
2111  5.892037
1828  5.892037
1256  5.892037
3335  5.892037
230   5.892037

[980 rows x 1 columns]


count    3918.000000
mean        5.892037
std         0.871254
min         3.000000
25%         5.000000
50%         6.000000
75%         6.000000
max         9.000000
Name: quality, dtype: float64

Na początku, jako próbę kontrolną, wykorzystamy jedynie wyraz wolny.  
Obserwujemy, że dla każdego rekordu została przypisana wartość średnia jakości ze zbioru treningowego.

Pomimo prymitywności tego modelu, okazał się on zaskakująco skuteczny - otrzymaliśmy błąd średniokwadratowy mniejszy od 1 - w przybliżeniu 0.886. Wynika to ze słabego zbalansowania badanego zbioru - jak wcześniej wspominaliśmy bardzo mało jest tu win o skrajnych jakościach. Jest to na ogół zgodne z intuicją - trudno jest stworzyć produkt bliski perfekcji, jak również żadna szanująca się firma nie chce sprzedawać win ocenianych jako najgorsze na rynku. Stąd zapewne ilościowa tendencja w danych do średniej jakości.

Sugeruje to również, że gdy brak nam doświadczenia czy odpowiedniego modelu, strzał że wino które właśnie otwieramy będzie średniej jakości ma spore szane się sprawdzić, albo przynajmniej nie powinniśmy pomylić się znacząco.

In [127]:
# const, alcohol
i += 1
considered_features.append(hypothesis_features[i])
test_results.append(describe_multiple_regression_model(X_train, X_test, y_train, y_test, considered_features, hypothesis_features[i]))

print(pd.DataFrame(test_results[i][1], index=y_test.index, columns=["y_pred"]).sort_values(by="y_pred"))

Added: alcohol
	Avg train error =	0.610666786267788
	Avg validate error =	0.6166589110158853
	Test error =		0.7306442340192554

        y_pred
3265  5.108744
294   5.264247
867   5.264247
4014  5.264247
3420  5.264247
...        ...
3086  6.881477
3083  6.881477
3915  6.974779
3150  6.974779
3904  6.974779

[980 rows x 1 columns]


Dodanie do modelu stężenia alkoholu jako cechy decyzyjnej zwiększyło zarówno zakres (pomiędzy max a min jest teraz prawie 2 różnicy) jak i dokładność wyników (zarówno przy treningu jak i testach o około 0.15 (rms)). Jednak model wciąż musi poszerzyć zakres wyników, by móc uzyskać lub chociaż zbliżyć się do wartości granicznych.

In [128]:
# const, alcohol, volatile acidity
i += 1
considered_features.append(hypothesis_features[i])
test_results.append(describe_multiple_regression_model(X_train, X_test, y_train, y_test, considered_features, hypothesis_features[i]))

print(pd.DataFrame(test_results[i][1], index=y_test.index, columns=["y_pred"]).sort_values(by="y_pred"))

Added: volatile acidity
	Avg train error =	0.5730409489874573
	Avg validate error =	0.5750177189933506
	Test error =		0.6866714897604801

        y_pred
1951  4.366818
2154  4.513727
294   4.635631
230   4.665712
4619  4.704887
...        ...
3150  6.787668
4645  6.793441
3086  6.848356
3083  6.848356
2814  6.859376

[980 rows x 1 columns]


Poprzez dodanie cechy "volatile acidity" do modelu udało się poszerzyć znacząco dolny zakres (oraz tak jak poprzednio zmniejszyć błąd, ale tym razem już o około 0.05).
Jest to więc zdecydowanie wartościowa zmiana, co jest zgodne z oczekiwaniami - jest to cecha u której zaobserwowaliśmy korelację z jakością, oraz brak korelacji ze stężeniem alkoholu, który to został dodany już do modelu. Zatem lotna kwasowość ma na model wpływ niepokrywający się z wpływem alkoholu. Jednak wciąż nie udało się sięgnąć granicznych wartości z danych testowych.

In [129]:
# const, alcohol, volatile acidity, density
i += 1
considered_features.append(hypothesis_features[i])
test_results.append(describe_multiple_regression_model(X_train, X_test, y_train, y_test, considered_features, hypothesis_features[i]))

print(pd.DataFrame(test_results[i][1], index=y_test.index, columns=["y_pred"]).sort_values(by="y_pred"))

# const, volatile acidity, density
describe_multiple_regression_model(X_train, X_test, y_train, y_test, ['const', 'volatile acidity', 'density'], "density, removed alcohol")
pass

Added: density
	Avg train error =	0.568105701666267
	Avg validate error =	0.5665895047926881
	Test error =		0.6837200879145863

        y_pred
1951  4.403961
294   4.568438
4619  4.615174
230   4.645602
2154  4.677024
...        ...
3500  6.891752
3086  6.895834
3083  6.895834
2781  6.922634
3150  6.925576

[980 rows x 1 columns]
Added: density, removed alcohol
	Avg train error =	0.6528045947721773
	Avg validate error =	0.6620605779249871
	Test error =		0.7981522226486801



Dodanie do modelu atrybutu gęstości przyczyniło się do wzrostu jakości rzędu mniejszego niż poprzednio, bo tylko 0.003. Oznacza to, że błędnym było założenie reprezentacji przez gęstość pozostałych cech, równoważących się z alkoholem. Również pozbycie się z modelu alkoholu na rzecz gęstości spowodowało znaczący spadek jakości, zatem jest ona na ogół raczej słabym reprezentantem jakości wina.

In [130]:
# const, alcohol, volatile acidity, density, alcohol_log
i += 1
considered_features.append(hypothesis_features[i])
test_results.append(describe_multiple_regression_model(X_train, X_test, y_train, y_test, considered_features, hypothesis_features[i]))

print(pd.DataFrame(test_results[i][1], index=y_test.index, columns=["y_pred"]).sort_values(by="y_pred"))

Added: alcohol_log
	Avg train error =	0.5598930122693325
	Avg validate error =	0.5578681456376386
	Test error =		0.6824533511240435

        y_pred
1951  4.170298
2154  4.428696
4619  4.702256
230   4.712467
294   4.780159
...        ...
3904  7.082939
3514  7.114153
3083  7.250267
3086  7.250267
3150  7.333846

[980 rows x 1 columns]


Po sprawdzeniu cech zadanych w hipotezie, postanowiliśmy sprawdzić jeszcze pewną wariację odnośnie jednej z jej cech. Mianowicie zdecyfowaliśmy się zniwelować prawoskośność wykresu alkoholu poprzez dodanie cechy zlogarytmowanej wartości stężenia alkoholu.

Choć wynikowy zysk jakości był marginalny, znacząco poszerzerzył się zakres wyników, co oznacza większą na ogół większą zdolność modelu do przyjmowania bardziej rozproszonych od średniej danych.

In [131]:
hypothesis_features = ['const', 'alcohol', 'volatile acidity', 'density', 'alcohol_log']
considered_features = []

min_lambda = 0.003
lambda_multiplier_step = 2
max_lambda = 150

best_results = []

for i, feature in enumerate(hypothesis_features):

    considered_features.append(feature)
    X_train_chosen = X_train[considered_features]

    current_lambda = min_lambda
    best_result = {'best_lambda': 0.00001, 'avg_train_error': 9999, 'avg_val_error': 9999, 'test_error': 9999, 'test_predicted': None}

    while current_lambda < max_lambda:

        ridge_model = Ridge(alpha=current_lambda, fit_intercept=False)
        X_train_scaled = standard_scaled(X_train_chosen, True)

        (avg_train_error, avg_val_error) = cross_validate(ridge_model, X_train_scaled, y_train)
        ridge_model.fit(X_train_scaled, y_train)
        test_predicted = ridge_model.predict(standard_scaled(X_test[considered_features], True))
        test_error = round_result(y_test, test_predicted)[1]['actual_mean_sqrt']

        if test_error < best_result['test_error']:
            best_result['best_lambda'] = current_lambda
            best_result['avg_train_error'] = avg_train_error
            best_result['avg_val_error'] = avg_val_error
            best_result['test_error'] = test_error
            best_result['test_predicted'] = test_predicted

        current_lambda *= lambda_multiplier_step

    best_results.append(best_result)
    
    print(f"""Added: {feature} (lambda={best_result['best_lambda']})
    \tAvg train error =\t{best_result['avg_train_error']}
    \tAvg validate error =\t{best_result['avg_val_error']}
    \tTest error =\t\t{best_result['test_error']}
    \tTest error gain =\t{test_results[i][0] - best_result['test_error']}
    """)

Added: const (lambda=49.152)
    	Avg train error =	0.7664878521630156
    	Avg validate error =	0.7699424034908015
    	Test error =		0.8813832757869122
    	Test error gain =	0.0049797906012517545
    
Added: alcohol (lambda=49.152)
    	Avg train error =	0.6189826446907183
    	Avg validate error =	0.6244869050264605
    	Test error =		0.7265834331392841
    	Test error gain =	0.004060800879971294
    
Added: volatile acidity (lambda=49.152)
    	Avg train error =	0.5813728819130781
    	Avg validate error =	0.5825860825494514
    	Test error =		0.6826376071866341
    	Test error gain =	0.004033882573845959
    
Added: density (lambda=49.152)
    	Avg train error =	0.5766039120286803
    	Avg validate error =	0.575227268830109
    	Test error =		0.6782744784691089
    	Test error gain =	0.00544560944547734
    
Added: alcohol_log (lambda=49.152)
    	Avg train error =	0.577230141455302
    	Avg validate error =	0.5758812913259181
    	Test error =		0.6784588821515019
    	Test error

Finalnie skorzystaliśmy z regresji grzbietowej, by możliwie zwiększyć zdolności modelu do znajdywania rozwiązań mniej skojarzonych z konkretnym zbiorem uczącym. Jednak ostateczny zysk w postaci mniejszego błędu testowego rms był stosunkowo niewielki - na poziomie około 0.005.

Jak widzimy poniżej, również zakresy zostały zawężone. Ale być może właśnie to skupienie bliżej średniej pozwala lepiej dopasowywać się do nieznanych danych?

In [133]:
print(pd.DataFrame(best_results[4]['test_predicted'], index=y_test.index, columns=["y_pred"]).sort_values(by="y_pred"))

        y_pred
1951  4.378749
294   4.518549
4619  4.576753
230   4.594193
2154  4.620189
...        ...
3500  6.789741
2814  6.794228
3083  6.830161
3086  6.830161
3150  6.847046

[980 rows x 1 columns]
